In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Brain
def ReadData(user = None, dataset = None):
  """
  Input:
    user (str): Who is using the notebook.
    dataset (str): Which data set are you using? "brain","lung" or "breast"
  """
  if user.lower() == 'jackie':
    root = '/content/drive/MyDrive/AUTOMATION/DarthImputer(AutomationProject)'
  elif user.lower() == 'thomas':
    root = '/content/drive/MyDrive/Colab Notebooks/DarthImputer(AutomationProject)'
  elif user.lower() == 'gabe':
    raise NotImplementedError('Gabe put your path here. Brazil numero uno!!!! hue hue hue hue.')
  else:
    raise Exception("Ya fucked up son. Who are you?")

  if dataset.lower() == 'brain':
    path = root + '/data/brain/Brain_GSE50161.csv'
  elif dataset.lower() == 'breast':
    path = root + '/data/breast/Breast_GSE45827.csv'
  elif dataset.lower() == 'lung':
    path = root + '/data/lung/clean/complete_dataframe.csv'
  else:
    raise Exception("I don't have that dataset you ding dong.")

  data = pd.read_csv(path)
  return root, data

In [2]:
root, brain_df = ReadData('thomas', 'brain')
# brain_df = pd.read_csv('../data/brain/Brain_GSE50161.csv')

In [ ]:
import sys
sys.path.append(f'{root}/src')

In [11]:
import helpers
from basic_impute import *
import mice

In [27]:
import importlib
importlib.reload(mice)
importlib.reload(helpers)

<module 'helpers' from '/Users/thomaszhang/Development/Python/02750_Automation_2024/project/src/helpers.py'>

# 1. Random masking to generate missing data

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [16]:
all_labels = helpers.EncodeLabels(brain_df['type'].to_numpy())
masked_data = helpers.Implement_Random_Masking(brain_df.iloc[:, 2:].to_numpy(), 0.3, 10, 2024)

In [28]:
mice_res = mice.ImputeDataMice(masked_data, seed = 2024)

[ 2494  7816  7977 30244 35456 47643 49335 51808 52351 52730]
Iteration: 0


100%|██████████| 10/10 [00:06<00:00,  1.57it/s]


Sum of deltas: 197.39004819825803. Max delta: 2.7506793005364116
Iteration: 1


100%|██████████| 10/10 [00:05<00:00,  1.67it/s]


Sum of deltas: 0.1175342911833197. Max delta: 0.002008186139626389
Iteration: 2


...

Sum of deltas: 0.11929023417779572. Max delta: 0.002888030715519463
Iteration: 8


100%|██████████| 10/10 [00:05<00:00,  1.71it/s]


Sum of deltas: 0.10680809187020213. Max delta: 0.0027163507359437844
Iteration: 9


100%|██████████| 10/10 [00:05<00:00,  1.67it/s]

Sum of deltas: 0.11592222874228586. Max delta: 0.0019471342262651348
Finished!


In [ ]:
# def Implement_Random_Masking(data, masking_portion):
#   '''
#   This function takes X and the masking portion and return the new X with the same shape with missing the given portion of random data
#   Args:
#   - data: full data
#   - Masking portion: float
#   Returns:
#   - masked_data: same shape as given data, missing given portion of data
#   '''
#   mask_indices = np.random.rand(*data.shape) < masking_portion
#   masked_data = data.copy()
#   masked_data[mask_indices] = np.nan

#   return masked_data

In [ ]:
brain_X = brain_df.drop(['samples', 'type'], axis=1).to_numpy()
brain_Y_original = brain_df['type'].to_numpy()

In [ ]:
encoder = LabelEncoder()
encoder.fit(all_labels)
brain_Y_encoded = encoder.transform(brain_Y_original)

In [ ]:
brain_X.shape, brain_Y_encoded.shape

((130, 54675), (130,))

In [ ]:
brain_X_train, brain_X_test, brain_Y_train, brain_Y_test = train_test_split(brain_X, brain_Y_encoded, test_size=0.2, random_state=42)

# Base learner

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import mode

# 1. Full dataset

In [ ]:
rf_clf0 = RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
rf_clf0.fit(brain_X_train, brain_Y_train)

RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
pred_0 = rf_clf0.predict(brain_X_test)

In [ ]:
score_0 = accuracy_score(pred_0, brain_Y_test)

In [ ]:
score_0

0.8846153846153846

# 2. 10% masking

In [ ]:
np.random.seed(42)

In [ ]:
X_train_1 = Implement_Random_Masking(brain_X_train, 0.1)

## 2.1. Mean Imputation

In [ ]:
def Imputation_Mean(X):
  '''
  Implement imputation
  '''
  X_imputed = X.copy()
  for col in np.arange(X_imputed.shape[1]):
    col_mean = np.nanmean(X_imputed[:, col])
    X_imputed[np.isnan(X_imputed[:, col]), col] = col_mean
  return X_imputed

In [ ]:
X_imputed_1 = Imputation_Mean(X_train_1)

### 2.1.2. Error compared to the original dataset

In [ ]:
diff1 = mean_squared_error(X_imputed_1, brain_X_train)
diff1

0.050901821703508106

### 2.1.3. Accuracy on test data

In [ ]:
rf_clf1 = RandomForestClassifier(n_estimators=10, random_state=42)
rf_clf1.fit(X_imputed_1, brain_Y_train)
pred_1 = rf_clf1.predict(brain_X_test)
score_1 = accuracy_score(pred_1, brain_Y_test)
score_1

0.8846153846153846

## 2.2. Mode Imputation

In [ ]:
def Imputation_Mode(X):
  '''
  Implement imputation
  '''
  X_imputed = X.copy()
  for col in np.arange(X_imputed.shape[1]):
    unique_values, counts = np.unique(X_imputed[:,col], return_counts=True)
    reverse_sorted_counts = np.argsort(counts)[::-1]
    col_mode = unique_values[0] if unique_values[0] != np.nan else unique_values[1]
    X_imputed[np.isnan(X_imputed[:, col]), col] = col_mode
  return X_imputed

In [ ]:
unique_values, counts = np.unique([np.nan, np.nan, 1,2,3,2,2], return_counts=True)
unique_values, counts

(array([ 1.,  2.,  3., nan]), array([1, 3, 1, 2]))

In [ ]:
np.argsort(counts)[::-1]

array([1, 3, 2, 0])

In [ ]:
X_imputed_mode1 = Imputation_Mode(X_train_1)

### 2.2.2. Error compared to the original dataset

In [ ]:
diff_mode1 = mean_squared_error(X_imputed_mode1, brain_X_train)
diff_mode1

0.2838962411981033

### 2.2.3. Accuracy on test data

In [ ]:
rf_clf1 = RandomForestClassifier(n_estimators=10, random_state=42)
rf_clf1.fit(X_imputed_mode1, brain_Y_train)
pred_mode_1 = rf_clf1.predict(brain_X_test)
score_mode_1 = accuracy_score(pred_1, brain_Y_test)
score_mode_1

0.8846153846153846

In [ ]:
int(3/2)

1